# Import libraries and dataset

In [133]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

# Import Datasets
df_raw = pd.read_csv('/Users/samlai/Documents/Vehicle_insurance_claim_project/data/raw/fraud_oracle.csv')

# Data Cleaning

## Drop useless features, change data type and impute strange values

In [134]:
# Drop ID Column
df_cleaned = df_raw.drop(columns = ['PolicyNumber'])

In [135]:
# RepNumber should be categorical
# WeekOfMonth and WeekOfMonthClaimed should be categorical
# Deductible should be categorical
# Year should be categorical
# DriverRating should be categorical
df_cleaned['RepNumber'] = df_cleaned['RepNumber'].astype('category')
df_cleaned['WeekOfMonth'] = df_cleaned['WeekOfMonth'].astype('category')
df_cleaned['WeekOfMonthClaimed'] = df_cleaned['WeekOfMonthClaimed'].astype('category')
df_cleaned['Deductible'] = df_cleaned['Deductible'].astype('category')
df_cleaned['Year'] = df_cleaned['Year'].astype('category')
df_cleaned['DriverRating'] = df_cleaned['DriverRating'].astype('category')

In [136]:
# MonthClaimed中有一個樣本為0
# 由於大部分的MonthClaimed與Month的值相同，因此替換成Month裡的'Jul'
df_cleaned.loc[df_cleaned['MonthClaimed'] == '0', 'MonthClaimed'] = 'Jul'
df_cleaned['MonthClaimed'].value_counts()

MonthClaimed
Jan    1446
May    1411
Mar    1348
Oct    1339
Jun    1293
Feb    1287
Nov    1285
Apr    1271
Sep    1242
Jul    1226
Dec    1146
Aug    1126
Name: count, dtype: int64

## Encode Categorical Data

### Encode Binary Variables (AccidentArea, Sex, Fault, PoliceReportFiled, WitnessPresent, AgentType)

In [137]:
from sklearn.preprocessing import LabelEncoder

binary_columns = [col for col in df_cleaned.columns if len(df_cleaned[col].unique()) == 2 and
                  col != 'FraudFound_P']

le = LabelEncoder()
for col in binary_columns:
    # Store original values before encoding
    original_values = df_cleaned[col].unique()
    df_cleaned[col] = le.fit_transform(df_cleaned[col])
    # Print mapping for each binary column
    print(f"\nMapping for {col}:")
    for i, label in enumerate(le.classes_):
        print(f"{label} -> {i}")


Mapping for AccidentArea:
Rural -> 0
Urban -> 1

Mapping for Sex:
Female -> 0
Male -> 1

Mapping for Fault:
Policy Holder -> 0
Third Party -> 1

Mapping for PoliceReportFiled:
No -> 0
Yes -> 1

Mapping for WitnessPresent:
No -> 0
Yes -> 1

Mapping for AgentType:
External -> 0
Internal -> 1


### Encode Ordinal Categorical Variables

In [138]:
for col in ['DriverRating', 'PastNumberOfClaims', 'AgeOfVehicle', 'NumberOfSuppliments']:
    print(f'Unique Values in {col}: {df_cleaned[col].unique()}')
    print('-'*100)

Unique Values in DriverRating: [1, 4, 3, 2]
Categories (4, int64): [1, 2, 3, 4]
----------------------------------------------------------------------------------------------------
Unique Values in PastNumberOfClaims: ['none' '1' '2 to 4' 'more than 4']
----------------------------------------------------------------------------------------------------
Unique Values in AgeOfVehicle: ['3 years' '6 years' '7 years' 'more than 7' '5 years' 'new' '4 years'
 '2 years']
----------------------------------------------------------------------------------------------------
Unique Values in NumberOfSuppliments: ['none' 'more than 5' '3 to 5' '1 to 2']
----------------------------------------------------------------------------------------------------


In [139]:
# DriverRating is already numerical, don't need to encode
PastNumberClaims_label = {'none': 0, '1': 1, '2 to 4': 2, 'more than 4': 3}
AgeOfVehicle_label = {'new': 0, '2 years': 1, '3 years': 2, '4 years': 3, '5 years': 4,
                      '6 years': 5, '7 years': 6, 'more than 7': 7}
NumberOfSuppliments_label = {'none': 0, '1 to 2': 1, '3 to 5': 2, 'more than 5': 3}

df_cleaned['PastNumberOfClaims'] = df_cleaned['PastNumberOfClaims'].map(PastNumberClaims_label)
df_cleaned['AgeOfVehicle'] = df_cleaned['AgeOfVehicle'].map(AgeOfVehicle_label)
df_cleaned['NumberOfSuppliments'] = df_cleaned['NumberOfSuppliments'].map(NumberOfSuppliments_label)

### Encode Nominal Categorical Variables

EDA中發現VehiclePrice, Deductible, Age of Policy Holder, Number of Cars, AddressChangeClaim與與詐欺率之間不是簡單的線性關係，因此用One-Hot encoding較適合（可以獨立學習每個類別）

In [140]:
# Encode Nominal Variables
df = pd.get_dummies(df_cleaned, drop_first = True)
df.shape

(15420, 138)

# 將特徵分組

In [141]:
# 保險人特徵
policyholder_feature = [
    'Sex', 'Age', 'DriverRating_2', 'DriverRating_3', 'DriverRating_4',
    'AgeOfPolicyHolder_18 to 20', 'AgeOfPolicyHolder_21 to 25',
    'AgeOfPolicyHolder_26 to 30', 'AgeOfPolicyHolder_31 to 35', 
    'AgeOfPolicyHolder_36 to 40', 'AgeOfPolicyHolder_41 to 50',
    'AgeOfPolicyHolder_51 to 65', 'AgeOfPolicyHolder_over 65',
    'AddressChange_Claim_2 to 3 years', 'AddressChange_Claim_4 to 8 years',
    'AddressChange_Claim_no change', 'AddressChange_Claim_under 6 months', 'NumberOfCars_2 vehicles',
    'NumberOfCars_3 to 4', 'NumberOfCars_5 to 8', 'NumberOfCars_more than 8',
    'PastNumberOfClaims', 'MaritalStatus_Married', 'MaritalStatus_Single',
    'MaritalStatus_Widow'
]

# 車禍事件特徵 
accident_feature = [
    'AccidentArea', 'Fault', 'PoliceReportFiled', 'NumberOfSuppliments',
    'Days_Policy_Accident_more than 30', 'Days_Policy_Accident_15 to 30',
    'Days_Policy_Accident_8 to 15', 'Days_Policy_Accident_none',
    'Days_Policy_Claim_8 to 15', 'Days_Policy_Claim_more than 30',
    'Days_Policy_Claim_none', 'BasePolicy_Collision', 'BasePolicy_Liability',
    'Deductible_400', 'Deductible_500', 'Deductible_700',
    'PolicyType_Sedan - Collision', 'PolicyType_Sedan - Liability',
    'PolicyType_Sport - Collision', 'PolicyType_Sport - All Perils',
    'PolicyType_Sport - Liability', 'PolicyType_Utility - All Perils',
    'PolicyType_Utility - Collision', 'PolicyType_Utility - Liability', 'WitnessPresent'
]

# 車輛特徵
vehicle_feature = [
    'AgeOfVehicle', 'VehiclePrice_30000 to 39000',
    'VehiclePrice_40000 to 59000', 'VehiclePrice_less than 20000', 'VehiclePrice_60000 to 69000',
    'VehiclePrice_more than 69000', 'Make_Chevrolet', 'Make_Ford',
    'Make_Honda', 'Make_Mazda', 'Make_Pontiac', 'Make_Toyota', 'Make_VW',
    'Make_BMW', 'Make_Dodge', 'Make_Ferrari', 'Make_Jaguar', 'Make_Lexus',
    'Make_Mecedes', 'Make_Mercury', 'Make_Nisson', 'Make_Porche', 'Make_Saab',
    'Make_Saturn', 'VehicleCategory_Sport', 'VehicleCategory_Utility'
]

# 時間特徵
time_feature = [
    'Month_Aug', 'Month_Dec', 'Month_Feb', 'Month_Jan', 'Month_Jul',
    'Month_Jun', 'Month_Mar', 'Month_May', 'Month_Nov', 'Month_Oct',
    'Month_Sep', 'WeekOfMonth_2', 'WeekOfMonth_3', 'WeekOfMonth_4',
    'WeekOfMonth_5', 'DayOfWeek_Monday', 'DayOfWeek_Saturday',
    'DayOfWeek_Sunday', 'DayOfWeek_Thursday', 'DayOfWeek_Tuesday',
    'DayOfWeek_Wednesday', 'Year_1995', 'Year_1996',
    'DayOfWeekClaimed_Friday', 'DayOfWeekClaimed_Monday', 'DayOfWeekClaimed_Saturday',
    'DayOfWeekClaimed_Sunday', 'DayOfWeekClaimed_Thursday', 'DayOfWeekClaimed_Tuesday',
    'DayOfWeekClaimed_Wednesday', 'MonthClaimed_Aug', 'MonthClaimed_Dec', 'MonthClaimed_Feb', 
    'MonthClaimed_Jan', 'MonthClaimed_Jul', 'MonthClaimed_Jun', 'MonthClaimed_Mar', 
    'MonthClaimed_May', 'MonthClaimed_Nov', 'MonthClaimed_Oct', 'MonthClaimed_Sep', 'WeekOfMonthClaimed_2',
    'WeekOfMonthClaimed_3', 'WeekOfMonthClaimed_4', 'WeekOfMonthClaimed_5'
]

# 其他特徵
other_feature = [
    'AgentType', 'RepNumber_2', 'RepNumber_3', 'RepNumber_4', 'RepNumber_5',
    'RepNumber_6', 'RepNumber_7', 'RepNumber_8', 'RepNumber_9',
    'RepNumber_10', 'RepNumber_11', 'RepNumber_12', 'RepNumber_13',
    'RepNumber_14', 'RepNumber_15', 'RepNumber_16'
]

Target = ['FraudFound_P']


In [142]:
# 檢查是否有特徵未被分組
all_grouped_features = policyholder_feature + accident_feature + vehicle_feature + time_feature + other_feature
all_features = df.columns.tolist()


ungrouped_features = [f for f in all_features if f not in all_grouped_features]
if ungrouped_features:
    print("未分組的特徵:")
    for f in ungrouped_features:
        print(f)
else:
    print("所有特徵都已被分組")

未分組的特徵:
FraudFound_P


In [143]:
# Split train test dataset
# 後續只用 train dataset 進行交叉驗證
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
df_train, df_test = train_test_split(
    df,
    test_size=0.2, 
    random_state=42,
    stratify=df['FraudFound_P']  
)


In [144]:
df_policyholder = df_train[policyholder_feature]
df_accident = df_train[accident_feature]
df_vehicle = df_train[vehicle_feature]
df_time = df_train[time_feature]
df_other = df_train[other_feature]
df_target = df_train[Target]

# Create a Function to evaluate created features

In [145]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import confusion_matrix

# Calculate the accuracy, precision, recall, and F1 score
def get_test_metrics(y_test, y_pred, y_prob):
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    auc = roc_auc_score(y_test, y_prob)

    print(f'Accuracy: {accuracy:.4f}')
    print(f'Precision: {precision:.4f}')
    print(f'Recall: {recall:.4f}')
    print(f'F1 Score: {f1:.4f}')
    print(f'AUC: {auc:.4f}')

    # Calculate and print confusion matrix
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8,6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label') 
    plt.show()

In [146]:
from sklearn.model_selection import StratifiedKFold
from imblearn.under_sampling import RandomUnderSampler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def cv_evaluation(df_policyholder, df_accident, df_vehicle, df_time, df_other, df_target):
    # 合併所有特徵
    df_new = pd.concat([df_policyholder, df_accident, df_vehicle, df_time, df_other, df_target], axis=1)
    X = df_new.drop(columns=['FraudFound_P'])
    y = df_new['FraudFound_P']
    
    # 定義 5 折交叉驗證
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    
    # 儲存每折的分數
    cv_scores = []
    
    # 進行交叉驗證
    for train_idx, val_idx in cv.split(X, y):
        # 分割數據
        X_train_fold = X.iloc[train_idx]
        y_train_fold = y.iloc[train_idx]
        X_val_fold = X.iloc[val_idx]
        y_val_fold = y.iloc[val_idx]
        
        # 下採樣
        rus = RandomUnderSampler(random_state=42)
        X_train_rus, y_train_rus = rus.fit_resample(X_train_fold, y_train_fold)
        
        # 訓練模型
        model = RandomForestClassifier(n_estimators=100, random_state=42)
        model.fit(X_train_rus, y_train_rus)
        
        # 預測
        y_val_pred = model.predict(X_val_fold)
        y_val_prob = model.predict_proba(X_val_fold)[:, 1]
        
        # 計算分數
        scores = {
            'accuracy': accuracy_score(y_val_fold, y_val_pred),
            'precision': precision_score(y_val_fold, y_val_pred),
            'recall': recall_score(y_val_fold, y_val_pred),
            'f1': f1_score(y_val_fold, y_val_pred),
            'auc': roc_auc_score(y_val_fold, y_val_prob)
        }
        cv_scores.append(scores)
    
    # 計算平均分數
    print("Average CV scores:")
    for metric in ['accuracy', 'precision', 'recall', 'f1', 'auc']:
        mean_score = np.mean([s[metric] for s in cv_scores])
        print(f"{metric}: {mean_score:.4f}")


# Feature Engineering

## 1. 保留 'AgeOfPolicyHolder'，刪除'Age'

In [147]:
df_policyholder_dropAge = df_policyholder.drop(columns = 'Age')
cv_evaluation(df_policyholder_dropAge, df_accident, df_vehicle, df_time, df_other, df_target)

Average CV scores:
accuracy: 0.6548
precision: 0.1387
recall: 0.9146
f1: 0.2408
auc: 0.8142


In [148]:
# 移除Age後，成效有些為提升，因此將Age移除
df_policyholder = df_policyholder.drop(columns = 'Age')

## 2.創造高風險車子品牌標籤特徵 (保留新建的高風險特徵，移除所有Brand Make特徵)

In [149]:
# Create HighRisk_brand feature based on specific makes
# Mecedes, BMW, Saab, Saturn, BMW, Ford, Mercury 這些為高風險品牌
# Accurca 是One-hot Encoding後的基準類別，其他 Make features若皆為 0 就是Accurca
high_risk_makes = ['Make_Mecedes', 'Make_Saab', 'Make_Saturn', 'Make_BMW', 'Make_Ford', 'Make_Mercury']

# Get all make columns
all_makes = ['Make_Chevrolet', 'Make_Ford', 'Make_Honda', 'Make_Mazda', 'Make_Pontiac', 
            'Make_Toyota', 'Make_VW', 'Make_BMW', 'Make_Dodge', 'Make_Ferrari', 
            'Make_Jaguar', 'Make_Lexus', 'Make_Mecedes', 'Make_Mercury', 'Make_Nisson',
            'Make_Porche', 'Make_Saab', 'Make_Saturn']

# Create HighRisk_brand feature
df_vehicle['HighRisk_brand'] = ((df_vehicle[high_risk_makes].any(axis=1)) | 
                               (df_vehicle[all_makes].sum(axis=1) == 0)).astype(int)


/var/folders/th/klmxbc1d2x58bny86xqxsjqc0000gn/T/ipykernel_82331/1558674046.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vehicle['HighRisk_brand'] = ((df_vehicle[high_risk_makes].any(axis=1)) |


In [150]:
cv_evaluation(df_policyholder, df_accident, df_vehicle, df_time, df_other, df_target)

Average CV scores:
accuracy: 0.6513
precision: 0.1370
recall: 0.9106
f1: 0.2382
auc: 0.8225


In [151]:
# 試著把Make相關features都刪掉，因為有HighRisk_brand了
df_vehicle_withoutMake = df_vehicle.drop(columns = all_makes)
cv_evaluation(df_policyholder, df_accident, df_vehicle_withoutMake, df_time, df_other, df_target)

Average CV scores:
accuracy: 0.6595
precision: 0.1389
recall: 0.9011
f1: 0.2406
auc: 0.8213


In [152]:
# 將18個Brand Make相關feature都刪除後，recall跟F1 score反而有些為提升，表示一個HighRisk_brand就可以提供足夠資訊
# 更新df_vehicle，移除 Brand Make 相關欄位
df_vehicle = df_vehicle.drop(columns = all_makes)

## 3.移除PolicyType 或BasePolicy

In [153]:
# PolicyType Column為Based Policy與Vehicle Category的組合，移除PolictType測試
# Get all PolicyType columns
policy_type_cols = [col for col in df_accident.columns if col.startswith('PolicyType')]

# Remove PolicyType columns from df_accident
df_accident_dropPolicyType = df_accident.drop(columns=policy_type_cols)

cv_evaluation(df_policyholder, df_accident_dropPolicyType, df_vehicle, df_time, df_other, df_target)

Average CV scores:
accuracy: 0.6555
precision: 0.1378
recall: 0.9038
f1: 0.2391
auc: 0.8191


In [154]:
# 移除Based Policy與Vehicle Category
# Get all Based Policy columns
BasePolicy_cols = [col for col in df_accident.columns if col.startswith('BasePolicy')]
df_accident_dropBasePolicy = df_accident.drop(columns=BasePolicy_cols)

VehicleCategory_col = [col for col in df_vehicle.columns if col.startswith('VehicleCategory')]
df_vehicle_dropCategory = df_vehicle.drop(columns=VehicleCategory_col)

cv_evaluation(df_policyholder, df_accident_dropBasePolicy, df_vehicle_dropCategory, df_time, df_other, df_target)

Average CV scores:
accuracy: 0.6610
precision: 0.1379
recall: 0.8875
f1: 0.2387
auc: 0.8207


In [155]:
df_accident = df_accident.drop(columns=BasePolicy_cols)
df_vehicle = df_vehicle.drop(columns=VehicleCategory_col)

## 4.'Days_Policy_Accident' and 'Days_Policy_Claim' Columns

In [156]:
# 部分類別樣本數較少，將其合併
# Create Days_Policy_Accident_morethan30 feature
days_policy_accident_col = [col for col in df_accident.columns if col.startswith('Days_Policy_Accident')]
df_accident = df_accident.drop(columns=days_policy_accident_col)
df_accident['Days_Policy_Accident_morethan30'] = (df_raw['Days_Policy_Accident'] == 'more than 30').astype(int)


In [157]:
# Create Days_Policy_Claim_morethan30 feature
days_policy_claim_col = [col for col in df_accident.columns if col.startswith('Days_Policy_Claim')]
df_accident = df_accident.drop(columns=days_policy_claim_col)
df_accident['Days_Policy_Claim_morethan30'] = (df_raw['Days_Policy_Claim'] == 'more than 30').astype(int)

In [158]:
cv_evaluation(df_policyholder, df_accident, df_vehicle, df_time, df_other, df_target)

Average CV scores:
accuracy: 0.6599
precision: 0.1365
recall: 0.8794
f1: 0.2363
auc: 0.8185


In [159]:
# 因為兩個特徵非常相似，試著刪掉其中一個
df_accident_dropAccident = df_accident.drop(columns = 'Days_Policy_Accident_morethan30')
cv_evaluation(df_policyholder, df_accident_dropAccident, df_vehicle, df_time, df_other, df_target)
print('-'*100)
df_accident_dropClaim = df_accident.drop(columns = 'Days_Policy_Claim_morethan30')
cv_evaluation(df_policyholder, df_accident_dropClaim, df_vehicle, df_time, df_other, df_target)

Average CV scores:
accuracy: 0.6616
precision: 0.1370
recall: 0.8781
f1: 0.2370
auc: 0.8175
----------------------------------------------------------------------------------------------------
Average CV scores:
accuracy: 0.6599
precision: 0.1366
recall: 0.8794
f1: 0.2365
auc: 0.8180


In [160]:
# drop 'Days_Policy_Claim_morethan30'
df_accident = df_accident.drop(columns = 'Days_Policy_Claim_morethan30')

## 5. 合併AddressChange_Claim, Number of Cars中的少數別

In [161]:
# 將AddressChange_Claim中的'under 6 months', '1 year', '2 to 3 years'合併為 'under 3 years'
# 但由於只剩 'under 3 years', 'no change', '4 to 8 years'三個選項，
# 因此透過'no change', '4 to 8 years'就可以推斷出'under 3 years'欄位的值，
# 不需要創造 'AddressChange_Claim_under 3 years'欄位
df_policyholder = df_policyholder.drop(columns = ['AddressChange_Claim_2 to 3 years', 'AddressChange_Claim_under 6 months'])

In [162]:
df_policyholder['More than 1 car'] = (df_raw['NumberOfCars'] != '1 vehicle').astype(int)
NumberOfCars_col = [col for col in df_policyholder.columns if col.startswith('NumberOfCars')]
df_policyholder = df_policyholder.drop(columns=NumberOfCars_col)

## 6. 創造高風險保險代理人特徵

In [163]:
df_other.columns

Index(['AgentType', 'RepNumber_2', 'RepNumber_3', 'RepNumber_4', 'RepNumber_5',
       'RepNumber_6', 'RepNumber_7', 'RepNumber_8', 'RepNumber_9',
       'RepNumber_10', 'RepNumber_11', 'RepNumber_12', 'RepNumber_13',
       'RepNumber_14', 'RepNumber_15', 'RepNumber_16'],
      dtype='object')

In [164]:
# RepNumber中有許多類別，創造HighRisk_RepNumber可以減少dummy variables的數量
RepNumber_col = [col for col in df_other.columns if col.startswith('RepNumber')]
df_other = df_other.drop(columns=RepNumber_col)

HighRisk_mask = [6, 7, 9, 10, 13]
df_other['HighRisk_RepNumber'] = df_raw['RepNumber'].isin(HighRisk_mask).astype(int)

In [165]:
cv_evaluation(df_policyholder, df_accident, df_vehicle, df_time, df_other, df_target)

Average CV scores:
accuracy: 0.6597
precision: 0.1391
recall: 0.9024
f1: 0.2410
auc: 0.8226


## 7. 創造高風險自付金額特徵

In [166]:
Deductible_col = [col for col in df_accident.columns if col.startswith('Deductible')]
df_accident = df_accident.drop(columns=Deductible_col)
df_accident['HiskRisk_DeductibleAMT'] = df_raw['Deductible'].isin([300, 500]).astype(int)

## 8. Time related features

In [167]:
# DayOfWeekClaimed中Saturday及Sunday的樣本數較少，合併成Weekend
# 合併後會有Monday, Tuesday, Wednesday, Thrusday, Friday, Weekend六個feature
# 由於其中5個feature已可提供所有資訊，故將Weekend刪除（Sunday與Saturday)
df_time = df_time.drop(columns = ['DayOfWeekClaimed_Saturday', 'DayOfWeekClaimed_Sunday'])

In [168]:
cv_evaluation(df_policyholder, df_accident, df_vehicle, df_time, df_other, df_target)

Average CV scores:
accuracy: 0.6636
precision: 0.1390
recall: 0.8903
f1: 0.2405
auc: 0.8184


# Export processed dataset

In [170]:
data_processed = pd.concat([df_policyholder, df_accident, df_vehicle, df_time, df_other, df_target], axis = 1)
data_processed.shape

(12336, 87)

In [171]:
# 對test dataset套用相同的處理
df_test = df_test.drop(columns = 'Age')

df_test['HighRisk_brand'] = ((df_test[high_risk_makes].any(axis=1)) | 
                               (df_test[all_makes].sum(axis=1) == 0)).astype(int)

df_test = df_test.drop(columns = all_makes)

df_test = df_test.drop(columns=BasePolicy_cols)
df_test = df_test.drop(columns=VehicleCategory_col)

df_test = df_test.drop(columns=days_policy_accident_col)
df_test['Days_Policy_Accident_morethan30'] = (df_raw['Days_Policy_Accident'] == 'more than 30').astype(int)

df_test = df_test.drop(columns=days_policy_claim_col)
df_test['Days_Policy_Claim_morethan30'] = (df_raw['Days_Policy_Claim'] == 'more than 30').astype(int)

df_test= df_test.drop(columns = 'Days_Policy_Claim_morethan30')

df_test = df_test.drop(columns = ['AddressChange_Claim_2 to 3 years', 'AddressChange_Claim_under 6 months'])

df_test['More than 1 car'] = (df_raw['NumberOfCars'] != '1 vehicle').astype(int)
df_test = df_test.drop(columns=NumberOfCars_col)

df_test = df_test.drop(columns=RepNumber_col)
df_test['HighRisk_RepNumber'] = df_raw['RepNumber'].isin(HighRisk_mask).astype(int)

df_test = df_test.drop(columns=Deductible_col)
df_test['HiskRisk_DeductibleAMT'] = df_raw['Deductible'].isin([300, 500]).astype(int)

df_test = df_test.drop(columns = ['DayOfWeekClaimed_Saturday', 'DayOfWeekClaimed_Sunday'])

df_test.shape

(3084, 87)

In [34]:
data_processed = pd.concat([data_processed, df_test], axis=0)
data_processed.to_csv('../data/processed/data_processed.csv', index = False)